In [1]:
import cv2
import os
import pandas as pd
from PIL import Image
from IPython.display import display
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelBinarizer, LabelEncoder
import tensorflow as tf
import numpy as np

2022-12-22 00:55:25.530165: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Crear dataframes de train y test através de las carpetas de fotos

dataset = []

for label in os.listdir('dataset'):
    if not os.path.isdir(f'dataset/{label}'):
        continue
    for image_filename in os.listdir(f'dataset/{label}'):
        image = Image.open(f'dataset/{label}/{image_filename}')
        pixels = np.array(image).astype(float)
        dataset.append((pixels, label))
            
# Convertir la llista de dataset en dataframe

df = pd.DataFrame(dataset, columns=['pixels', 'label'])


In [3]:
df.head()

,pixels,label
0,"[[108.0, 83.0, 63.0, 65.0, 89.0, 111.0, 121.0,...",happy
1,"[[137.0, 142.0, 159.0, 162.0, 158.0, 134.0, 11...",happy
2,"[[111.0, 148.0, 155.0, 167.0, 181.0, 191.0, 19...",happy
3,"[[151.0, 156.0, 121.0, 100.0, 80.0, 116.0, 151...",happy
4,"[[5.0, 4.0, 5.0, 9.0, 10.0, 9.0, 10.0, 12.0, 8...",happy


In [4]:
df['label'].unique()

array(['happy', 'sad', 'fear', 'surprise', 'neutral', 'angry', 'disgust'],
      dtype=object)

In [3]:
# Asignar 'X' y 'y', height i width

X = df['pixels'].values
y = df['label'].values

In [4]:
# Normalize data

def pross_img(x):
    return x/255
X = list(map(pross_img, X))

In [5]:
lb = LabelBinarizer()
y = lb.fit_transform(df['label'].values)
print(y)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]]


In [6]:
# Dividir en train y test 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
print(len(X_train))
print(len(y_train))

print(set([len(x) for x in X_train]))
print(set([len(x) for x in y_train]))

28709
28709
{48}
{7}


In [ ]:
# Definir Modelo
width=48
height=48
nlabels=7

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28709, width, height)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(nlabels, activation='softmax')
])


# Compilar los datos
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

2022-12-22 00:56:48.115624: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
# Train
model.fit(X_train, y_train, batch_size=32, epochs=10)